In [1]:
addprocs(11)

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [2]:
# import FunctionalCollections
import Iterators
import Pipe
import Compat
import JLD
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
@everywhere using Compat
@everywhere using JLD

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [3]:
@everywhere fold_indexes=load("brown_glove_folds.jld","fold_indexes")

@everywhere function fold_split(fold_ii, raw_bow_res)
    ground_sents = Vector{ASCIIString}[rset[1] for rset in raw_bow_res]
    reconstructed_bows = Vector{ASCIIString}[rset[2] for rset in raw_bow_res]
    
    test_indexes = fold_indexes[fold_ii]
    training_indexes = trues(ground_sents)
    training_indexes[test_indexes]=false

    test_unordered_sents = reconstructed_bows[test_indexes]
    test_ground = ground_sents[test_indexes]
    training_sents = ground_sents[training_indexes]
    test_unordered_sents,test_ground, training_sents
end
    

In [4]:
@everywhere typealias S ASCIIString
@everywhere typealias State{T} Tuple{T,T}

import PyCall
@everywhere using PyCall
#http://www.nltk.org/howto/probability.html
@everywhere @pyimport nltk
@everywhere @pyimport nltk.probability as nltk_prob
@everywhere const START_MARKER1 = "**START1**"
@everywhere const START_MARKER2 = "**START2**"
@everywhere const END_MARKER1 = "**END1**"
@everywhere const END_MARKER2 = "**END2**"

@everywhere function train_language_model{T}(train_corpus::Vector{Vector{T}})
    function py_collect(xs::PyObject)
        xst = []
        for x in xs
            push!(xst,x)
        end
        xst
    end
    function trigram_buffer(sent)
        [START_MARKER1, START_MARKER2, sent..., END_MARKER1, END_MARKER2] 
    end

    training_trigrams = vcat([py_collect(nltk.trigrams(trigram_buffer(sent))) for sent in train_corpus]...)
    kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, training_trigrams))
    
    function trigram_model(given1::S, given2::S, event::S)
        kn_prob_dist[:prob]((given1, given2, event))
    end
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: Use

In [5]:
import FixedLengthPriorityQueues
@everywhere using FixedLengthPriorityQueues

@everywhere typealias NodeType Tuple{Float64, FunctionalCollections.AbstractList{S}, State{S}, Vector{S}}
@everywhere function best_order(unordered_words::Vector{S}, languauge_model::Function; mem_limit=23^31)
    _lm = Dict{Tuple{S,S,S},Float64}()
    function cached_lm(w1,w2,w3)
        get!(_lm, (w1,w2,w3)) do
            languauge_model(w1,w2,w3)
        end
    end
    function transition_prob(cur_state::State{S}, next_word::S)
            cached_lm(cur_state[1],cur_state[2], next_word)
    end
  
    _get_options = Dict{Tuple{State, Vector{S}}, Any}()
    function get_options(cur_state::State{S}, remaining_words)
        get!(_get_options, (cur_state,remaining_words)) do
            if length(remaining_words)==0
                #Final Step -- leaf node
                tp = transition_prob(cur_state, END_MARKER1)
                # Given P(END_MARKER2 | curstatep[2]==END_MARKER1) = 1.0
                # Do not need to consider P(END_MARKER2 | curstatep[2]==END_MARKER1, curstatep[1])
                next_state = (cur_state[2],END_MARKER1)
                return [(tp, next_state, S[])]
            end
            #Otherwise -- nonfinal step, get all childreen
            Task() do
                for ii in 1:length(remaining_words)
                    next_word = remaining_words[ii]
                    tp = transition_prob(cur_state, next_word)           
                    if tp>0.0
                        still_remaining_words = sub(remaining_words,[1:ii-1; ii+1:length(remaining_words)])
                        next_state = (cur_state[2],next_word)
                        produce(tp, next_state, still_remaining_words)
                    end
                end
            end
        end
    end
                
    
    initial_state = (START_MARKER1, START_MARKER2)
    initial_node::NodeType = (1.0, EmptyList{S}(), initial_state, unordered_words)
    
    pending = FixedLengthPriorityQueue(NodeType, Float64, mem_limit, Base.Order.Reverse)     
    enqueue!(pending, initial_node, 1.0)
    
    #At end of search pending contains all the partway steps that have not yet been explored fully
    #But that can't be more optimal than the returned solution, we don't do anything with them right now
    while(true)
        (sofar_prob, sofar_words, state, remaining_words) = dequeue!(pending)
        
        is_terminal = state[2]==END_MARKER1
        if is_terminal
            @assert length(remaining_words)==0
            return sofar_words|> tail |> collect |> reverse!, sofar_prob #Note: We take the tail of the sofar words before reversing to cut off then end marker
        end
        
        #Nontermial, ie has children, even if it is just the single terminal child
        for (tp, next_state, still_remaining_words) in get_options(state, remaining_words)
            @assert(tp>0.0)
            total_next_prob = sofar_prob*tp
            next_sofar_words = cons(next_state[2], sofar_words)
            new_node::NodeType = (total_next_prob, next_sofar_words, next_state, still_remaining_words)
            enqueue!(pending, new_node, total_next_prob)
        end
        if length(pending)==0
            #All Paths failed, return the best we could do so far, with the unsortable on the end
            sorted = sofar_words |> collect |> reverse!
            return ([sorted...; remaining_words...], length(sorted)*1.0)
        end
    end 
end

INFO: Precompiling module IntervalHeaps...


In [ ]:
#Prepare Oracle Results -- no regen
#test_set = load("results/data/brown_glove300.jld")["corpus"]
#oracle_res = pmap(test_set, err_stop=true) do target_sent
#    score=0.0
#    sol = shuffle(target_sent)
#    (target_sent, sol, score)
#end
#@save "results/bags/oracle_res.jld" oracle_res

In [6]:
@everywhere const len_cap = 18
@everywhere raw_bow_res = load("results/bags/oracle_res.jld", "oracle_res")

function process_fold(fold_ii)
    test_bows, test_ground_order, train = fold_split(fold_ii,raw_bow_res)

    #Avoid "serialising a pointer" by getting each process to create their own copy of Language model
    #This also means that the language models are independent (as under the hood they are not readonly, readomg them changes them as they have cache)
    r_language_models =Dict([pid=>remotecall(train_language_model,pid, train) for pid in workers()])
    pmap(test_ground_order, test_bows, err_stop=true) do ground_order, bow
        lm = fetch(r_language_models[myid()])
        generated_order, prob = if length(ground_order)<=len_cap
            best_order(bow, lm; mem_limit=1000)
        else
            (bow, NaN)
        end
        (ground_order, generated_order, prob)
    end
end

jldopen("results/ordered/oracle_ordered_vA1000.jld", "w") do file
    for fold_ii in 1:length(fold_indexes)
        res = process_fold(fold_ii)
        write(file, "fold_$(fold_ii)", res)
        println("fold $fold_ii done")
    end
end


 in depwarn at deprecated.jl:73
 in remotecall at deprecated.jl:50
 in process_fold at In[6]:9
 in anonymous at In[6]:23
 in jldopen at /home/ubuntu/.julia/v0.5/JLD/src/JLD.jl:240
 [inlined code] from essentials.jl:114
 in include_string at loading.jl:355
 in execute_request_0x535c5df2 at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:177
 [inlined code] from dict.jl:723
 in eventloop at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:429
while loading In[6], in expression starting on line 21
 in depwarn at deprecated.jl:73
 in remotecall at deprecated.jl:50
 in process_fold at In[6]:9
 in anonymous at In[6]:23
 in jldopen at /home/ubuntu/.julia/v0.5/JLD/src/JLD.jl:240
 [inlined code] from essentials.jl:114
 in include_string at loading.jl:355
 in execute_request_0x535c5df2 at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:177
 [inlined code] from dict.jl:723
 in eventloop at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:141
 in anonymous at t

fold 1 done
fold 2 done
fold 3 done
fold 4 done
fold 5 done
fold 6 done
fold 7 done
fold 8 done
fold 9 done
fold 10 done


In [ ]:
@pyimport nltk.corpus as nltk_corpus
corpus_reader=nltk_corpus.brown
corpus = Vector{ASCIIString}[[lowercase(word) for word in sent] for sent in (corpus_reader[:sents]()|> collect)]
@everywhere const lm::Function = train_language_model(corpus)

gc()
test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a","fine","comedy", "."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy", "."])
#test_bag =  shuffle(["it", "is", "so", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "good", "."])
#test_bag =  shuffle(["no", "way", "."])
#test_bag =  shuffle(["no", "."])
@time best_order(test_bag, lm, mem_limit=1000)